# 0. Configuration

In [1]:
#TODO - add formulae in line with functions

# 1. Modules and functions

In [2]:
import numpy as np
from math import log2

# 2. Main

## 2.1. Precision@K

In [6]:
def precission_at_k(y_true: np.array, y_pred: np.array, k: int) -> float:
    """
    y_true: true labels
    y_pred: predicted lables
    k: cutoff length
    """

    if sum(y_true) == 0:
        return -1

    argsort = np.argsort(y_pred)[::-1]
    y_true_sorted = y_true[argsort]
    true_positives = y_true_sorted[:k].sum()

    return true_positives / k


In [7]:
# example array
y_true = np.array([1, 0, 0, 1, 0, 0])
y_pred = np.array([6, 2, 3, 5, 4, 1])

In [8]:
argsort = np.argsort(y_pred)[::-1]
argsort

array([0, 3, 4, 2, 1, 5])

In [9]:
y_true_sorted = y_true[argsort]
y_true_sorted

array([1, 1, 0, 0, 0, 0])

In [16]:
true_positives = y_true_sorted[:6].sum()
true_positives

2

In [17]:
# as expected
precission_at_k(y_true, y_pred, k = 6)

0.3333333333333333

## 2.2. AP@K, MAP@K

In [20]:
def average_precision(y_true: np.array, y_pred: np.array, k: int) -> float:

    if sum(y_true) == 0:
        return -1

    if len(y_pred) > k:
        y_pred = y_pred[:k]

    argsort = np.argsort(y_pred)[::-1]
    y_true_sorted = y_true[argsort]

    num_hits = 0
    score = 0

    for i, p in enumerate(y_true_sorted, 1):
        if p == 1:
            num_hits += 1
            score += num_hits / i
    if num_hits == 0:
        output = 0
    
    else:
        output = score / min(len(y_true), k)

    return output


In [10]:
average_precision(y_true, y_pred, k = 3)

0.3333333333333333

## MAP@K

In [30]:
def mean_average_precision(y_true: dict , y_pred: dict, k: int) -> float:
    ap = 0
    for i in y_pred.keys():
        AP = average_precision(y_true[i], y_pred[i], k)
        ap += AP
    output = ap/len(y_true)

    return output
    

In [39]:
y_true = {1: np.array([0,0,1,1,0,1]),
          2: np.array([1,1,0,0,0,0])}

y_pred = {1: np.array([1,2,4,5,7,3]),
          2: np.array([4,3,5,3,7,8])}

MAP = mean_average_precision(y_true, y_pred, 3)
MAP

0.36111111111111105

Let's check whether function works correctly:

In [42]:
apk1 = average_precision(y_true[1], y_pred[1], 3)
apk2 = average_precision(y_true[2], y_pred[2], 3)

print(apk1, apk2)
(apk1 + apk2) / 2

0.3333333333333333 0.38888888888888884


0.36111111111111105

## 2.3. MRR

In [43]:
def reciprocal_rank(y_true: np.array, y_pred: np.array) -> float:
    
    argsort = np.argsort(y_pred)[::-1]
    y_true_sorted = y_true[argsort]
    for i, val in enumerate(y_true_sorted, 1):     
        if val == 1:
            return 1 / i
    return 0


In [44]:
# example array for MRR
y_true = np.array([1, 0, 0, 1, 0, 0])
y_pred = np.array([0, 2, 3, 3.5, 4, 1])

In [45]:
reciprocal_rank(y_true, y_pred)

0.5

## 2.4. NDCG

In [49]:
def compute_gain(y_value: float, gain_scheme: str) -> float:
    
    gain = {'exp2': 2 ** y_value - 1,
            'const': y_value}

    return float(gain[gain_scheme])

In [47]:
def dcg(y_true: np.array, y_pred: np.array, gain_scheme: str) -> float:
    
    dcg = 0
    argsort = np.argsort(y_pred)[::-1]
    y_true_sorted = y_true[argsort]

    for idx, val in enumerate(y_true_sorted, 1):
        gain = compute_gain(val, gain_scheme)
        dcg += gain / log2(idx + 1)
        
    return dcg


NDCG with cutoff:

In [50]:
def ndcg(y_true: np.array, ys_pred: np.array, k: int, gain_scheme: str = 'const')  -> float:
    
    #cutoff condition
    if len(ys_pred)>k:
        ys_pred = ys_pred[:k]
    
    # pred dcg then we calc the same to find max possible
    preds_dcg = dcg(y_true, ys_pred, gain_scheme)
    max_possible_dcg = dcg(y_true, y_true, gain_scheme)

    return preds_dcg / max_possible_dcg


In [72]:
y_pred = np.array([6, 5, 4, 3, 2, 1]) # some score to sort
y_true = np.array([3, 2, 3, 0, 1, 2])

In [81]:
ndcg(y_true, y_pred, 6, 'exp2')

0.9488107485678985

# TODO
- Write MAP@K function; <b> Done! </b>
- Modify ndcg() such that it incomporates cutoff param K  <b> Done! </b>